In [17]:
import numpy as np
import string
import pandas as pd
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.model_selection import ParameterGrid

In [18]:
train = pd.read_csv("hwk3_datasets/imdb-train.txt",delimiter='\t', dtype=str, header=None, names = ['Review','Score'])
test = pd.read_csv("hwk3_datasets/imdb-test.txt",delimiter='\t', dtype = str, header=None, names = ['Review','Score'])
val = pd.read_csv("hwk3_datasets/imdb-valid.txt",delimiter='\t', dtype = str, header = None, names = ['Review','Score'])

transtab = str.maketrans('','', string.punctuation)

train_review_len = np.zeros(len(train['Review']))
test_review_len = np.zeros(len(test['Review']))
val_review_len = np.zeros(len(val['Review']))

for i,ex in enumerate(train['Review']):
    train['Review'][i] = ex.translate(transtab)
    train['Review'][i] = (train['Review'][i]).lower()
for i,ex in enumerate(test['Review']):
    test['Review'][i] = ex.translate(transtab) 
    test['Review'][i] = test['Review'][i].lower()
for i,ex in enumerate(val['Review']):
    val['Review'][i] = ex.translate(transtab) 
    val['Review'][i] = val['Review'][i].lower()
    
train['Score']=train['Score'].astype(np.int32)
test['Score']=test['Score'].astype(np.int32)
val['Score']=val['Score'].astype(np.int32)
                 

In [19]:
vectorizer = CountVectorizer(max_features = 10000) 

train_vectors_notnorm = vectorizer.fit_transform(train['Review'])
test_vectors_notnorm = vectorizer.transform(test['Review'])
val_vectors_notnorm = vectorizer.transform(val['Review'])      

normalizer = Normalizer(norm='l2')

train_vectors = normalizer.transform(train_vectors_notnorm)
test_vectors = normalizer.transform(test_vectors_notnorm)
val_vectors = normalizer.transform(val_vectors_notnorm)

In [20]:
def eval_classifier(classifier):    
    try:
        classifier.fit(train_vectors, train['Score'])
        train_y =classifier.predict(train_vectors)
        test_y = classifier.predict(test_vectors)
        val_y = classifier.predict(val_vectors)
       
    except:
        classifier.fit(train_vectors.toarray(), train['Score'])
        train_y =classifier.predict(train_vectors.toarray())
        test_y = classifier.predict(test_vectors.toarray())
        val_y = classifier.predict(val_vectors.toarray())        
        
    train_f1 = f1_score(train['Score'],train_y, average='macro')
    val_f1 = f1_score(val['Score'],val_y, average='macro')
    test_f1 = f1_score(test['Score'],test_y, average='macro')
    
    print(type(classifier))
    print(f"Train F1: {train_f1}")
    print(f"Val F1: {val_f1}")
    print(f"Test F1: {test_f1}")
    print("\n")
    return val_f1

In [21]:
def eval_classifier_quick(classifier):    
    try:
        classifier.fit(train_vectors, train['Score'])
        val_y = classifier.predict(val_vectors)        
    except:
        classifier.fit(train_vectors.toarray(), train['Score'])
        val_y = classifier.predict(val_vectors.toarray())          
        
    val_f1 = f1_score(val['Score'],val_y, average='macro')    
    return val_f1
    
def test_classifier(classifier):    
    try:
        classifier.fit(train_vectors, train['Score'])
        test_y = classifier.predict(test_vectors)        
    except:
        classifier.fit(train_vectors.toarray(), train['Score'])
        test_y = classifier.predict(test_vectors.toarray())          
        
    test_f1 = f1_score(test['Score'],test_y, average='macro')    
    return test_f1

In [22]:
bayes_params = ParameterGrid({})
tree_params = ParameterGrid({'random_state':[10],'max_depth':[None,10,100,1000],'min_samples_split':[2,5,10]})
svm_params = ParameterGrid({'random_state':[10],'loss':['hinge','squared_hinge'],'C':[.5,1.0,2.0]})

classifiers= [(GaussianNB, bayes_params), (DecisionTreeClassifier, tree_params), (svm.LinearSVC, svm_params)]

In [25]:

def best_parameters(classifier, param_grid):
    best_Score=0
    best_params=None
    for params in param_grid:
        print(f"Trying: {params}")
        Score = eval_classifier_quick(classifier(**params))
        print(f"F1 Score: {Score}\n")
        if Score>best_Score:
            best_Score=Score
            best_params=params       
            
    print(f"Best params: {best_params}")
    print(f"Best F1 Score: {best_Score}\n")
    return classifier(**best_params)

In [26]:
for pair in classifiers:
    classifier = pair[0]
    param_grid = pair[1]
    print(classifier)    
    best_classifier = best_parameters(classifier,param_grid) 
    print(f"Test score for best params: {test_classifier(best_classifier)}\n")

<class 'sklearn.naive_bayes.GaussianNB'>
Trying: {}


MemoryError: 